In [73]:
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from pathlib import Path
from urlpath import URL
import json
import glob 
import numpy as np
import seaborn as sns
from skimage.measure import shannon_entropy
import string
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# https://pypi.org/project/pytesseract/
# sudo apt install tesseract-ocr
import pytesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

from prepareCSV import cleanOcrText
pd.options.display.max_columns = None

[nltk_data] Downloading package words to /home/lilatee/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lilatee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lilatee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/lilatee/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/lilatee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_

In [50]:
df = pd.read_csv("./GB_new_columns.csv")
df = df.loc[:, ["video_id", "thumbnail_link", 'region']]
region = df['region'][0]
df = df.set_index("video_id")
print(df.shape)
df[:3]

(3235, 2)


,thumbnail_link,region
video_id,,
Jw1Y-zhQURU,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,GB
3s1rvMFUweQ,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,GB
n1WpP7iowLc,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,GB


### Pobieranie thumbnaili

In [ ]:
def readJSONList(path):
    with open(path, 'r') as file:
        loaded_list = json.loads(file.read())
    return loaded_list

def writeJSONList(path, list_to_save):
    with open(path, 'w') as file:
        file.write(json.dumps(list_to_save))

def downloadThumbnails(df, quality='hq'):
    list_of_ids_without_thumbnail = []
    ids = df["video_id"].values
    urls = df["thumbnail_link"].values
    
    region = df['region'][0]
    Path("./data/thumbnailsHQ/" + region).mkdir(parents=True, exist_ok=True)
    savepath = Path("./data/thumbnailsHQ/" + region)
    filename = quality + 'default.jpg'
    
    for id, url in zip(ids, urls): 
        temp_url = URL(url)
        temp_url = temp_url.parent / filename
        
        try:
            img = io.imread(str(temp_url))
        except Exception:
            list_of_ids_without_thumbnail.append(id)
            print(f"{id} no thumbnail")
            continue
        io.imsave(savepath / (id + ".png") , img)
    
    writeJSONList(savepath / "ids_without_thumbnail.txt", list_of_ids_without_thumbnail)
        
    return list_of_ids_without_thumbnail

In [ ]:
# list_of_ids_without_thumbnail = downloadThumbnails(df)
df.loc[df.video_id=="0Yp9WS6cJFA",:]

### Średnia wartość kanałów w obrazie, entropia, ocr tesseract

In [3]:
%%time
# Odkomentować zapis jeśli trzeba!!!
imgs_path = Path(f'./data/thumbnailsHQ/{region}')

for i, img_path in enumerate(glob.glob(f"{imgs_path}/*.png")): 
    path = Path(img_path)
    img = io.imread(path)
    n_of_pixels = img.shape[0]*img.shape[1]
    
    intensity = np.sum(img, axis=0)
    mean_intensity_each_channel = np.sum(intensity, axis=0)/n_of_pixels
    df.loc[df.video_id == path.stem, "r"] = mean_intensity_each_channel[0]
    df.loc[df.video_id == path.stem, "g"] = mean_intensity_each_channel[1]
    df.loc[df.video_id == path.stem, "b"] = mean_intensity_each_channel[2]
    df.loc[df.video_id == path.stem, "entropy"] = shannon_entropy(img)
    df.loc[df.video_id == path.stem, "thumb_ocr_text"] = pytesseract.image_to_string(img)
    
    if (i % 100 == 0):
        print(i)

df["clean_thumb_ocr_text"] = df.apply(lambda row: cleanOcrText(row.thumb_ocr_text), axis=1)
df.to_csv(f"./our_data/etap2_{region}-tesseract.csv", index=False)
df[:3]

[nltk_data] Downloading package wordnet to /home/lilatee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/lilatee/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/lilatee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
cleanOcrText: 0
cleanOcrText: 100
cleanOcrText: 200
cleanOcrText: 300
cleanOcrText: 400
cleanOcrText: 500
cleanOcrText: 600
cleanOcrText: 700
cleanOcrText: 800
cleanOcrText: 900
cleanOcrText: 1000
cleanOcrText: 1100
cleanOcrText: 1200
cleanOcrText: 1300
cleanOcrText: 1400
cleanOcrText: 1500
cleanOcrText: 1600
cleanOcrText: 1700
cleanOcrText: 1800
cleanOcrText: 1900
cleanOcrText: 2000
cleanOcrText: 2100
cleanOcrText: 2200
cleanOcrText: 2300
cleanOcrText: 2400
cleanOcrText: 2500
cleanOcrText: 2600
cleanOcrText: 2700
cleanOcrText: 2800
cleanOcrText: 2900
cleanOcrText: 3000
cleanOcrText: 3100
cleanOcrText: 3200
CPU times: user 4min 14s, sys: 24.1 s, total: 4min 39s
Wall time: 11min


,video_id,thumbnail_link,region,r,g,b,entropy,thumb_ocr_text,clean_thumb_ocr_text
0,Jw1Y-zhQURU,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,
1,3s1rvMFUweQ,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,
2,n1WpP7iowLc,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,GB,66.50283,71.743704,79.699635,6.470819,\n,


In [65]:
df = pd.read_csv(f"./our_data/etap2_{region}-tesseract.csv")
df = df.set_index("video_id")

In [ ]:
ocr_videos = df.loc[df.clean_thumb_ocr_text != '',:][["video_id","thumb_ocr_text","clean_thumb_ocr_text"]]
ocr_videos.shape

### Wizualizacja OCRa

In [ ]:
ocr_videos = df.loc[df.clean_thumb_ocr_text != '',:][["video_id","thumb_ocr_text","clean_thumb_ocr_text"]][:10]

n_of_cols = 3
n_of_rows = int(ocr_videos.shape[0]/n_of_cols)
fig, axes = plt.subplots(n_of_rows, n_of_cols, figsize=(n_of_cols*5, n_of_rows*5))

for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        video = ocr_videos[i*n_of_cols+j:i*n_of_cols+j+1]
        video_id = video.video_id.values[0]
        path = Path(f'./data/thumbnailsHQ/{region}/{video_id}.png')
        
        ax.set_title(video.clean_thumb_ocr_text.values[0])
        img = io.imread(str(path))
        ax.imshow(img)

In [ ]:
ll = df.loc[df.clean_thumb_ocr_text != '',:][["thumb_ocr_text","clean_thumb_ocr_text"]].values
print(ll.shape)
print(df.shape)
# for el in ll:
#     print("="*100)
#     print(el[1])
#     print("="*100)
#     print(el[0])


### Lepszy OCR

#### [Rozpoznawanie napisów - https://github.com/clovaai/deep-text-recognition-benchmark](https://github.com/clovaai/deep-text-recognition-benchmark)

In [30]:
%run CRAFT-pytorch/test_modified.py \
    --trained_model="CRAFT-pytorch/craft_mlt_25k.pth" \
    --test_folder="data/thumbnailsHQ/{region}"\
    --cuda=False \
    --result_folder="data/OCR_result/{region}/"

#### Wycięcie fragmentów obrazów z napisami

In [31]:
from skimage import io
import numpy as np
from pathlib import Path
import glob

def genSlicesWithText(CRAFT_path_results=f"./data/OCR_result/{region}/", path_images=f"./data/thumbnailsHQ/{region}"):
    def bounding_box(points):
        x_coordinates, y_coordinates = zip(*points)
        return [(min(x_coordinates), min(y_coordinates)), (max(x_coordinates), max(y_coordinates))]

    CRAFT_path_results = Path(CRAFT_path_results)
    path_images = Path(path_images)

    for j, result_path in enumerate(glob.glob(str(CRAFT_path_results) + "/*.txt")):
        result_path = Path(result_path)
        video_id = result_path.stem[4:] # removing "res_" to obtain video_id
        
        img_path = Path(path_images / (video_id + ".png"))
        img = io.imread(img_path)
        shape = img.shape

        bboxes_path = CRAFT_path_results / ('res_' + img_path.stem + ".txt")
        bboxes = []
        with open(bboxes_path, "r") as bboxFile:
            for line in bboxFile.readlines():
                numbers = np.array(line.split(',')).astype(int)
                points = []

                for x,y in zip(numbers[::2], numbers[1::2]):
                    points.append((x, y))
                bboxes.append(bounding_box(points))


        if len(bboxes) > 0:
                Path(CRAFT_path_results / "words" / img_path.stem).mkdir(parents=True, exist_ok=True)
                img_dir = Path(CRAFT_path_results / "words" / img_path.stem)

        for i, bbox in enumerate(bboxes):

            img_slice = img[max(bbox[0][1], 0): min(bbox[1][1], shape[0]), max(bbox[0][0], 0):min(bbox[1][0], shape[1])]
            io.imsave(img_dir / f"{i}.png", img_slice)

    #     if j == 100:
    #         break
    
genSlicesWithText(CRAFT_path_results=f"./data/OCR_result/{region}/", path_images=f"./data/thumbnailsHQ/{region}")

#### [Wykrywanie napisów - https://github.com/clovaai/CRAFT-pytorch](https://github.com/clovaai/CRAFT-pytorch)

In [32]:
%run deep-text-recognition-benchmark/demo_modified.py \
--Transformation TPS \
--FeatureExtraction ResNet \
--SequenceModeling BiLSTM \
--Prediction Attn \
--image_folder "data/OCR_result/{region}/words/" \
--saved_model "deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth" \
--out_df_path "our_data/etap2_ocr_{region}.csv"

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth


### Dodanie kolumny z wynikami nowego OCRa

In [36]:
df_ocr = pd.read_csv(f"./our_data/etap2_ocr_{region}.csv")
df_ocr[:5]

,video_id,word_id,ocr_text
0,0-VAnh7r-_8,0,destinyt2
1,0-VAnh7r-_8,1,forsaken
2,0-VAnh7r-_8,2,3if
3,0ANDjPQPS_Q,0,thank
4,0ANDjPQPS_Q,1,you


In [71]:
df_ocr = df_ocr.groupby(by="video_id",axis=0)["ocr_text"].apply(lambda x: ' '.join(x)).reset_index()
df_ocr = df_ocr.set_index("video_id")
df = df.join(df_ocr).reset_index()
df = df.rename(columns={"ocr_text":"thumb_ocr_text_V2"})
df[:5]

,video_id,thumbnail_link,region,r,g,b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2
0,Jw1Y-zhQURU,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3s1rvMFUweQ,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,n1WpP7iowLc,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,GB,66.502830,71.743704,79.699635,6.470819,\n,NaN,wigy water an vevo
3,PUTEiSjKwJU,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,GB,86.534334,73.284271,69.150666,6.839139,\n\ney tderr ott\n\nSTE a CLASS OF 92\n...,class,3 10 3 92 salford city class of 92 4 dudley2n ...
4,rHwDegptbI4,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df["clean_thumb_ocr_text_V2"] = df.apply(lambda row: cleanOcrText(row.thumb_ocr_text_V2), axis=1)
df.to_csv(f"./our_data/etap2_{region}-V2.csv", index=False)
df[:10]

cleanOcrText: 0
cleanOcrText: 100
cleanOcrText: 200
cleanOcrText: 300
cleanOcrText: 400
cleanOcrText: 500
cleanOcrText: 600
cleanOcrText: 700
cleanOcrText: 800
cleanOcrText: 900
cleanOcrText: 1000
cleanOcrText: 1100
cleanOcrText: 1200
cleanOcrText: 1300
cleanOcrText: 1400
cleanOcrText: 1500
cleanOcrText: 1600
cleanOcrText: 1700
cleanOcrText: 1800
cleanOcrText: 1900
cleanOcrText: 2000
cleanOcrText: 2100
cleanOcrText: 2200
cleanOcrText: 2300
cleanOcrText: 2400
cleanOcrText: 2500
cleanOcrText: 2600
cleanOcrText: 2700
cleanOcrText: 2800
cleanOcrText: 2900
cleanOcrText: 3000
cleanOcrText: 3100
cleanOcrText: 3200


,video_id,thumbnail_link,region,r,g,b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2
0,Jw1Y-zhQURU,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,3s1rvMFUweQ,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,n1WpP7iowLc,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,GB,66.502830,71.743704,79.699635,6.470819,\n,NaN,wigy water an vevo,water
3,PUTEiSjKwJU,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,GB,86.534334,73.284271,69.150666,6.839139,\n\ney tderr ott\n\nSTE a CLASS OF 92\n...,class,3 10 3 92 salford city class of 92 4 dudley2n ...,city class burton
4,rHwDegptbI4,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,AumaWl0TNBo,https://i.ytimg.com/vi/AumaWl0TNBo/default.jpg,GB,95.003171,95.024786,91.284381,6.807186,\n,NaN,NaN,
6,2Vv-BfVoq4g,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,GB,50.475770,41.008709,45.347471,6.762013,\n,NaN,NaN,
7,LMCuKltaY3M,https://i.ytimg.com/vi/LMCuKltaY3M/default.jpg,GB,177.639190,177.110909,177.468981,2.602036,\n,NaN,elbow the best of,elbow best
8,9t9u_yPEidY,https://i.ytimg.com/vi/9t9u_yPEidY/default.jpg,GB,67.812957,45.861829,37.470625,6.237680,\n,NaN,vevo,
9,ONQ-fAp5X64,https://i.ytimg.com/vi/ONQ-fAp5X64/default.jpg,GB,151.588484,86.007512,86.608212,6.743056,\n,NaN,NaN,
